In [1]:
from selenium import webdriver
from time import sleep

from datetime import datetime, timedelta
from geopy import geocoders
from tzwhere import tzwhere
import pytz

tz = tzwhere.tzwhere()

tz_arg = pytz.timezone("America/Argentina/Buenos_Aires")
# tz_rus = pytz.timezone("Europe/Volgograd")
tz_rus = {
    'Moscow': 'Europe/Moscow',
    'Ekaterinburg': 'Asia/Yekaterinburg',
    'St. Petersburg': 'Europe/Moscow',
    'Sochi': 'Europe/Moscow',
    'Kazan': 'Europe/Moscow',
    'Saransk': 'Europe/Moscow',
    'Kaliningrad': 'Europe/Kaliningrad',
    'Samara': 'Europe/Samara',
    'Rostov-On-Don': 'Europe/Moscow',
    'Nizhny Novgorod': 'Europe/Moscow',
    'Volgograd': 'Europe/Volgograd' }

geocoder = geocoders.GoogleV3()  # Para buscar si es que me falta un timezone

In [2]:
d = webdriver.Chrome()
sleep(2)
d.maximize_window()

In [8]:
class Partido:
    def __init__(self, html):
        
        self.link = html.get_attribute('href')
        self.pid = int(self.link.split('/')[-2:-1][0])
        
        self.grupo = html.find_element_by_class_name("fi__info__group").text
        self.equipos = [x.text for x in html.find_elements_by_class_name("fi-t__nText ")]
        self.ciudad = html.find_element_by_class_name("fi__info__venue").text
        
        # Paso la ciudad a timezone
        if not self.ciudad in tz_rus:
            sleep(2)  # Para no saturar la API
            place, (lat, lng) = geocoder.geocode(self.ciudad)
            tz_rus[self.ciudad] = tz.tzNameAt(lat, lng)
            
        self.timezone = pytz.timezone(tz_rus[self.ciudad])
        # print(self.ciudad, self.timezone)
        
        fecha_aux = html.find_element_by_class_name("fi-mu__info__datetime").text
        self.fecha_local = self.timezone.localize(datetime.strptime(fecha_aux, "%d %b %Y - %H:%M Local time"))
        self.fecha = self.fecha_local.astimezone(tz_arg)
        # print(self.fecha_local, self.fecha)
        
        # No dependo de que actualicen fi-s__scoreText
        if self.fecha + timedelta(minutes=90) <= tz_arg.localize(datetime.now()):
            self.pendiente = False
            try:
                self.marcador = [int(x) for x in html.find_element_by_class_name("fi-s__scoreText").text.split('-')]
            except:
                self.marcador = [ -1, -1 ]
        else:
            self.pendiente = True
            
    def actualizarGoles(self):
        d.get(self.link)
        sleep(2)
        
        self.goles = { self.equipos[0]: [], self.equipos[1]: [] }
        for lista in d.find_elements_by_class_name("fi-mh__scorers__home"):
            for html in lista.find_elements_by_class_name("fi-mh__scorer"):
                gol = Gol(html, self.pid)
                self.goles[self.equipos[0]].append(gol)
                contarGoles(gol)  # De paso lo cuento para el torneo
        for lista in d.find_elements_by_class_name("fi-mh__scorers__away"):
            for html in lista.find_elements_by_class_name("fi-mh__scorer"):
                gol = Gol(html, self.pid)
                self.goles[self.equipos[1]].append(gol)
                contarGoles(gol)  # De paso lo cuento para el torneo
    
    def __repr__(self):
        if self.pendiente:
            return "%s [%s] %s - %s" % (self.fecha, self.grupo,
                                          self.equipos[0], self.equipos[1])
        else:
            return "%s [%s] %s %d - %s %d" % (self.fecha, self.grupo,
                                          self.equipos[0], self.marcador[0],
                                          self.equipos[1], self.marcador[1])

In [4]:
class Gol:
    def __init__(self, html, pid):
        self.pid = pid
        self.jid = int(html.find_element_by_tag_name('a').get_attribute('href').split('/')[-1])
        self.jugador = html.find_element_by_class_name("fi-p__n").text
        
        self.minuto = html.find_element_by_class_name("fi-mh__scorer__minute").text
        # self.minuto = int(self.minuto[:-1])  # Que onda 90'+1
        
        self.etiqueta = html.find_element_by_class_name("fi-mh__scorer__label").text
        
    def __repr__(self):
        if self.etiqueta:
            return "%s %s (%s)" % (self.jugador, self.minuto, self.etiqueta)
        else:
            return "%s %s" % (self.jugador, self.minuto)

In [5]:
goles_torneo = {}
def contarGoles(g):
    if not g.jugador in goles_torneo:
            goles_torneo[g.jugador] = []
    goles_torneo[g.jugador].append(g)

In [9]:
d.get('https://www.fifa.com/worldcup/matches/')
fixture = {}

for link in d.find_elements_by_class_name('fi-mu__link'):
    partido = Partido(link)
       
    # Compruebo que el partido esté terminado o en marcha
    if partido.pendiente:
        continue
        
    print(partido)
    
    
    if not partido.grupo in fixture:
        fixture[partido.grupo] = []
        
    fixture[partido.grupo].append(partido)

2018-06-14 12:00:00-03:00 [Group A] Russia 5 - Saudi Arabia 0
2018-06-15 09:00:00-03:00 [Group A] Egypt 0 - Uruguay 1
2018-06-15 12:00:00-03:00 [Group B] Morocco 0 - IR Iran 1
2018-06-15 15:00:00-03:00 [Group B] Portugal -1 - Spain -1


In [10]:
# Busco los goles
for grupo in fixture.keys():
    for partido in fixture[grupo]:
        partido.actualizarGoles()
        print(partido.goles)

TypeError: not all arguments converted during string formatting

In [ ]:
goles_torneo

In [ ]:
# Terminados
for html_partido in d.find_elements_by_class_name('result'):
    partido = Partido(html_partido)
    print(partido)
    
# Ongoing
for html_partido in d.find_elements_by_class_name('live'):
    partido = Partido(html_partido)
    print(partido)